In [1]:
import mediapipe as mp 
import cv2
import socket
import numpy as np


In [2]:
mp_hands=mp.solutions.hands
mp_drawings=mp.solutions.drawing_utils
mp_styles=mp.solutions.drawing_styles

In [3]:
def connect():
    mySocket = socket.socket()
    mySocket.bind(('localhost', 5000))
    mySocket.listen(5)
    conn , addr = mySocket.accept()
    print("device connected")
    return conn
# print(conn,addr)

In [4]:
Camera = cv2.VideoCapture(0)
Camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
Camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

mp_hands = mp.solutions.hands
conn = None
opened = 0
data = ""

# Setup socket connection outside of the loop
conn = connect()

while True:
    if conn is None:
        print("Waiting for connection...")
        conn = connect()

    if conn is not None:
        if opened == 0:
            Camera = cv2.VideoCapture(0)
            opened = 1

        ret, frame = Camera.read()

        if not ret:
            break

        flipped_image = cv2.flip(frame, 1)
        annotated_image = flipped_image.copy()
        image_height, image_width, _ = frame.shape

        with mp_hands.Hands(static_image_mode=False,
                            max_num_hands=1,
                            min_detection_confidence=0.6) as hands:
            results = hands.process(flipped_image)
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Only send data when hand landmarks are detected
                    data += str(hand_landmarks.landmark[8].x * image_width) + "," + str(
                        hand_landmarks.landmark[8].y * image_height) + ","
                    
                    msg = bytes(data, 'utf-8')
                    try:
                        conn.send(msg)
                        print(data)
                        data = ""
                    except:
                        conn = None
                        print("Device Disconnected. Waiting for reconnection...")

                    mp.solutions.drawing_utils.draw_landmarks(annotated_image,
                                                             hand_landmarks,
                                                             mp.solutions.hands.HAND_CONNECTIONS)

                    annotated_image = cv2.circle(annotated_image,
                                                (int(hand_landmarks.landmark[8].x * image_width),
                                                 int(hand_landmarks.landmark[8].y * image_height)),
                                                20, (0, 0, 255), 2)

        cv2.imshow("MediaPipe Hands", annotated_image)

        if cv2.waitKey(1) & 0xFF == 27:
            break

# Release resources
Camera.release()
cv2.destroyAllWindows()

device connected
335.8320999145508,-41.26449108123779,
184.79625701904297,65.0597620010376,
192.23718643188477,12.77625560760498,
212.97592163085938,-15.48574447631836,
212.0533561706543,-12.352924346923828,
215.4576873779297,-9.915211200714111,
212.32818603515625,-4.944448471069336,
210.111083984375,-9.835982322692871,
212.1129035949707,-5.270876884460449,
217.77910232543945,18.750600814819336,
211.8824005126953,30.364665985107422,
205.73360443115234,48.84894847869873,
207.10477828979492,51.043524742126465,
207.0402717590332,52.10859775543213,
209.56741333007812,61.25770568847656,
213.66331100463867,68.60278129577637,
216.29104614257812,89.1789436340332,
221.36140823364258,112.54241466522217,
221.03424072265625,120.25978088378906,
219.3414306640625,125.06598472595215,
218.8425636291504,130.14660358428955,
223.9234733581543,132.2505283355713,
218.97537231445312,137.5768232345581,
224.6609878540039,131.47054195404053,
224.19713973999023,131.26600742340088,
223.79657745361328,133.4508991